In [1]:
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
import string
import re
# From geeksforgeeks.org
# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

In [5]:
#Remove Numbers
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result 
  
input_str = "13000 people receive wildfires evacuation orders in california "
remove_numbers(input_str) 

' people receive wildfires evacuation orders in california '

In [6]:
#Remove English Stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download("stopwords")

def remove_stopwords(text): 
    stop_words = set(stopwords.words("english"))
    re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
    filtered_text =  re_stop_words.sub(" ", text)
    return filtered_text 
  
example_text = "raining flooding florida tampabay tampa 18 or 19 days ive lost count "
remove_stopwords(example_text) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tresyap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'raining flooding florida tampabay tampa 18  19 days ive lost count '

In [7]:
# Stemmer
from nltk.stem.porter import PorterStemmer 
stemmer = PorterStemmer() 
  
# stem words in the list of tokenised words 
def stem_words(text): 
    stems = [stemmer.stem(word) for word in text.split()] 
    return " ".join(stems)
  
text = "raining flooding florida tampabay tampa 18 or 19 days ive lost count "
stem_words(text) 

'rain flood florida tampabay tampa 18 or 19 day ive lost count'

In [8]:
def preprocess(df):
    try:
        df.drop(['keyword', 'location'], axis=1, inplace=True)
    except:
        print("columns already dropped")
    df.text = df.text.apply(remove_punctuation)
    df.text = df.text.apply(remove_numbers)
    df.text = df.text.apply(remove_stopwords)
    df.text = df.text.apply(stem_words)
    return df

In [9]:
# preprocess(data)

In [10]:
nlp = spacy.load('en_core_web_lg')
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(doc).vector for doc in tqdm(data.text)])

100%|█████████████████████████████████████████████████████████████████████████████| 7613/7613 [01:04<00:00, 117.96it/s]


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X_train, X_val, y_train, y_val = train_test_split(doc_vectors, data.target, test_size=0.2, random_state=42)

In [12]:
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, dual=False, max_iter=10000)
model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [13]:
pred = model.predict(X_val)

In [14]:
f1_score(y_val, pred)

0.7508038585209003

In [ ]:
#Leaderboard Score: 0.791